In [1]:
# Requisites
from gensim.models import TfidfModel
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
import numpy as np
from scipy import spatial
from typing import Tuple, List
from datasets import load_dataset
import tensorflow as tf
from gensim.models import fasttext
from textsim_class import TextSimilarity
import spacy
from gensim.models import KeyedVectors
from gensim.models import word2vec
import pandas as pd

c:\Users\Usuario\Documents\Universitat\4rt Quatri\PLH\Practica4\word-embeddings\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("projecte-aina/sts-ca", trust_remote_code=True)

In [3]:
"""
from gensim.models import FastText
import gensim
# No funciona de moment
model = gensim.models.fasttext.FastTextKeyedVectors.load('./models/fasttext/cc_ca_300.bin')
"""

"\nfrom gensim.models import FastText\nimport gensim\n# No funciona de moment\nmodel = gensim.models.fasttext.FastTextKeyedVectors.load('./models/fasttext/cc_ca_300.bin')\n"

In [3]:
wv_model = word2vec.Word2Vec.load(f"./models/word2vec_complete.model")
wv_model = wv_model.wv

In [4]:
from gensim.models import KeyedVectors
# Word2vec permet dos formats: text i binari
kv_model = KeyedVectors.load_word2vec_format('./models/word2vec/34/model.bin', binary=True)

In [5]:
ft_model = fasttext.load_facebook_model('./models/fasttext/cc.ca.300.bin.gz')
ft_model = ft_model.wv
"""
Doesn't work
from gensim.models import FastText
ft_model = FastText.load('./models/fasttext/cc_ca_300.bin', mmap='r')
"""


"\nDoesn't work\nfrom gensim.models import FastText\nft_model = FastText.load('./models/fasttext/cc_ca_300.bin', mmap='r')\n"

In [6]:
model_spacy = spacy.load("ca_core_news_md")

In [9]:
model_roberta = spacy.load("ca_core_news_trf")

In [10]:
list_of_models = [
    # model     mode    pretr   cls remap   train   recalc  name
    (wv_model, "onehot", True, True, True, True,True,  "Onehot"),
    (wv_model, "mean",  True, True, True, True, True, "Word2vec propi mean"),
    (wv_model, "tfidf",  True, True, True, True, True,"Word2vec propi tfidf"),
    (kv_model, "mean",  True, True, True, True, True,"Word2vec nlpl mean"),
    (kv_model, "tfidf",  True, True, True, True, True,"Word2vec nlpl tfidf"),
    (ft_model, "mean",  True, True, True, True, True,"Word2vec ccca300 mean"),
    (ft_model, "tfidf",  True, True, True, True, True,"Word2vec ccca300 tfidf"),
    (model_spacy, "spacy",  True, True, True, True, True, "Spacy md"),
    (model_roberta, "roberta", True, False, True, True, "roberta_mean_mapped_pairs", "Roberta mean"),
    (model_roberta, "roberta", True, True, True, True, "roberta_mapped_pairs", "Roberta cls"),
    (wv_model, "embeddings", False, True, True, False, True, "Random notrain"),
    (wv_model, "embeddings", False, True, True, True, True, "Random"),
    (wv_model, "embeddings", True, True, True, False, True, "Initialized w2v propi notrain"),
    (wv_model, "embeddings", True, True, True, True, True, "Initialized w2v propi"),
    (ft_model, "embeddings", True, True, True, False,True,  "Initialized ccca300 notrain"),
    (ft_model, "embeddings", True, True, True, True, True, "Initialized ccca300")
]

In [19]:
results_dict: dict[str, dict[str, str|float|None]] = {
    model_name + str(exec_model): {
        'Model': model_name,
        'Exec model': exec_model,
        'Mode': None, 
        'Pretrained': None, 
        'CLS': None, 
        'Trained': None, 
        'Train pearson': None, 
        'Val pearson': None,
        'Test pearson': None
        } for model_name in [l[-1] for l in list_of_models] for exec_model in range(7)
}

In [20]:
total_models = len(list_of_models)
count = 0

In [23]:
for information in list_of_models:
    model, mode, pretrained, cls, remap, trainable, recalculate, name = information

    text_sim = TextSimilarity(model=model, dataset=dataset, mode=mode, pretrained=pretrained, cls=cls, remap=remap, trainable=trainable, recalculate=recalculate, dict_size=1000000)

    text_sim.define_model(id=0)
    pearson_train, pearson_val, pearson_test = text_sim.baseline_model()



    results_dict[name + str(6)] = {
                'Model': name,
            'Exec model': "Baseline",
            'Mode': mode, 
            'Pretrained': pretrained, 
            'CLS': cls, 
            'Trained': trainable, 
            'Train pearson': pearson_train, 
            'Val pearson': pearson_val,
            'Test pearson': pearson_test
        }
    
    for index in range(6):
        text_sim.define_model(id=index)
        pearson_train, pearson_val, pearson_test = text_sim.train(num_epochs=128)
        results_dict[name + str(index)] = {
                'Model': name,
            'Exec model': str(index),
            'Mode': mode, 
            'Pretrained': pretrained, 
            'CLS': cls, 
            'Trained': trainable, 
            'Train pearson': pearson_train, 
            'Val pearson': pearson_val,
            'Test pearson': pearson_test
        }
        print(f"Tensorflow model {index}/6 done")
    count += 1
    print(f"{count} models done of a total of {total_models}")

Epoch 1/128


c:\Users\Usuario\Documents\Universitat\4rt Quatri\PLH\Practica4\word-embeddings\.venv\Lib\site-packages\scipy\spatial\distance.py:622: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / np.sqrt(uu * vv)


33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.9786 - val_loss: 0.9261 - learning_rate: 0.0010
Epoch 2/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8967 - val_loss: 0.8753 - learning_rate: 0.0010
Epoch 3/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7930 - val_loss: 0.8448 - learning_rate: 0.0010
Epoch 4/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7282 - val_loss: 0.8293 - learning_rate: 0.0010
Epoch 5/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6802 - val_loss: 0.8175 - learning_rate: 0.0010
Epoch 6/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6409 - val_loss: 0.8036 - learning_rate: 0.0010
Epoch 7/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6024 - val_loss: 0.7959 - learning_rate: 0.0010
Epoch 8/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5713 - val_loss: 0.7884 - learning_rate: 0.0010
Epoch 9/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5476 - val_loss: 0.7782 - learning_rate: 0.0010
Epoch 10/128
33/33 ━━━━━━━━━━━━

c:\Users\Usuario\Documents\Universitat\4rt Quatri\PLH\Practica4\word-embeddings\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.7349 - val_loss: 0.7253 - learning_rate: 0.0010
Epoch 2/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6233 - val_loss: 0.7182 - learning_rate: 0.0010
Epoch 3/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4833 - val_loss: 0.7193 - learning_rate: 0.0010
Epoch 4/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3434 - val_loss: 0.7429 - learning_rate: 0.0010
Epoch 5/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2219 - val_loss: 0.7488 - learning_rate: 0.0010
Epoch 6/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1789 - val_loss: 0.7483 - learning_rate: 0.0010
Epoch 7/128
19/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1696 
Epoch 7: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1624 - val_loss: 0.7559 - learning_rate: 0.0010
Epoch 8/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1465 - val_loss: 0.7567 - learning_rate: 1.0000e-04
Epoc

...

33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - loss: 3.8768 - val_loss: 3.7241 - learning_rate: 0.0010
Epoch 2/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 2.4702 - val_loss: 3.3179 - learning_rate: 0.0010
Epoch 3/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 1.2348 - val_loss: 2.9571 - learning_rate: 0.0010
Epoch 4/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.7081 - val_loss: 2.9648 - learning_rate: 0.0010
Epoch 5/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.3881 - val_loss: 2.9514 - learning_rate: 0.0010
Epoch 6/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.2594 - val_loss: 2.9002 - learning_rate: 0.0010
Epoch 7/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.2304 - val_loss: 2.9117 - learning_rate: 0.0010
Epoch 8/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.1898 - val_loss: 2.9281 - learning_rate: 0.0010
Epoch 9/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.1806 - val_loss: 2.8657 - learning_rate: 0.0010
Epoch 10/128
33/33 ━━━

c:\Users\Usuario\Documents\Universitat\4rt Quatri\PLH\Practica4\word-embeddings\.venv\Lib\site-packages\keras\src\layers\layer.py:877: UserWarning: Layer 'sequential_64' (of type Sequential) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\Users\Usuario\Documents\Universitat\4rt Quatri\PLH\Practica4\word-embeddings\.venv\Lib\site-packages\keras\src\layers\layer.py:877: UserWarning: Layer 'exp_layer_45' (of type Exp_layer) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\Users\Usuario\Documents\Universitat\4rt Quatri\PLH\Practica4\word-embeddings\.venv\Lib\site-packages\keras\src\layers\layer.py:877: UserWarning: Layer 'exp_layer_44' (of type Exp_layer) was passed an input with a mask

33/33 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step - loss: 2.0530 - val_loss: 0.7903 - learning_rate: 0.0010
Epoch 2/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 1.5502 - val_loss: 0.7902 - learning_rate: 0.0010
Epoch 3/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 1.3221 - val_loss: 0.7901 - learning_rate: 0.0010
Epoch 4/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 1.1737 - val_loss: 0.7712 - learning_rate: 0.0010
Epoch 5/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 1.1230 - val_loss: 0.7557 - learning_rate: 0.0010
Epoch 6/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.9164 - val_loss: 0.7515 - learning_rate: 0.0010
Epoch 7/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.9918 - val_loss: 0.7458 - learning_rate: 0.0010
Epoch 8/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.9317 - val_loss: 0.7433 - learning_rate: 0.0010
Epoch 9/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.9007 - val_loss: 0.7429 - learning_rate: 0.0010
Epoch 10/128
33/33 ━━━

c:\Users\Usuario\Documents\Universitat\4rt Quatri\PLH\Practica4\word-embeddings\.venv\Lib\site-packages\keras\src\layers\layer.py:877: UserWarning: Layer 'sequential_66' (of type Sequential) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\Users\Usuario\Documents\Universitat\4rt Quatri\PLH\Practica4\word-embeddings\.venv\Lib\site-packages\keras\src\layers\layer.py:877: UserWarning: Layer 'exp_layer_47' (of type Exp_layer) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\Users\Usuario\Documents\Universitat\4rt Quatri\PLH\Practica4\word-embeddings\.venv\Lib\site-packages\keras\src\layers\layer.py:877: UserWarning: Layer 'exp_layer_46' (of type Exp_layer) was passed an input with a mask

33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 1.9897 - val_loss: 6.3647 - learning_rate: 0.0010
Epoch 2/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 1.7880 - val_loss: 6.3943 - learning_rate: 0.0010
Epoch 3/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 1.6457 - val_loss: 6.2795 - learning_rate: 0.0010
Epoch 4/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 1.6011 - val_loss: 5.9634 - learning_rate: 0.0010
Epoch 5/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 1.6164 - val_loss: 5.6637 - learning_rate: 0.0010
Epoch 6/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 1.5184 - val_loss: 4.8683 - learning_rate: 0.0010
Epoch 7/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 1.6455 - val_loss: 2.6781 - learning_rate: 0.0010
Epoch 8/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 1.5406 - val_loss: 3.9667 - learning_rate: 0.0010
Epoch 9/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 1.4515 - val_loss: 2.8720 - learning_rate: 0.0010
Epoch 10/128
33/33 ━━━

In [51]:
results_df = pd.DataFrame(results_dict).T

# Reorder the columns and sort the values by the 'Avg. Statistic' column
results_df = results_df[['Model', 'Exec model', 'Mode', 'Pretrained', 'CLS', 'Trained', 'Train pearson', 'Val pearson', 'Test pearson']]
results_df = results_df.sort_values(by='Test pearson', ascending=False)


results_df.to_csv('./results/similarity_results.csv', index=False)
print(f"Results saved to './results/similarity_results.csv'.")

Results saved to './results/similarity_results.csv'.


In [69]:
import matplotlib.pyplot as plt
import seaborn as sns
def generate_model_barplot(results_df: pd.DataFrame, metric: str, model = "all") -> None:
	"""
	Generate a barplot of the models evaluated for the given metric.

	Parameters
	----------
	results_df : pd.DataFrame
		The DataFrame containing the results of the evaluation.
	metric : str
		The metric to generate the barplot for. It can be one of ['Avg. Statistic', 'Pearson', 'Spearman', 'OOV Perc.'].
	family : str, optional
		The family of models to generate the barplot for. It can be one of ['all', 'w2v', 'ft']. The default is 'all'.
	"""
	assert metric in ['Train pearson', 'Val pearson', 'Test pearson'], f"Invalid metric '{metric}'. Must be one of ['Avg. Statistic', 'Pearson', 'Spearman', 'OOV Perc.']."
	# Filter the dataframe by the family of models

	if model != "all":
		results_filtered_df = results_df[results_df['Exec model']==model].copy()
	else:
		results_filtered_df = results_df.copy()

	# Sort the values by the metric (descending order, except for 'OOV Perc.')
	results_filtered_df = results_filtered_df.sort_values(by=metric, ascending=(metric == 'OOV Perc.'))

	# Remove .model from the model names
	results_filtered_df['Model'] = results_filtered_df['Model'].apply(lambda x: x.removesuffix('.model'))

	# Generate the barplot
	plt.figure(figsize=(10, 6))
	sns.barplot(x='Model', y=metric, data=results_filtered_df)
	plt.xticks(rotation=45, ha='right')
	plt.xlabel('Model')
	plt.ylabel(metric)
	plt.tight_layout()
	plt.savefig(f'./results/plots/textsim_{model}_{metric.lower().replace(".", "").replace(" ", "_")}.png')
	plt.close()

generate_model_barplot(results_df, "Test pearson", model="Baseline")

# Single model tests

In [47]:
a = TextSimilarity(model=wv_model, dataset=dataset, mode="embeddings", pretrained=False, cls=True, remap=True, trainable=True, dict_size=1000000, recalculate=True)

In [26]:
a.mode = "roberta"

In [48]:
a.define_model(id=1)

c:\Users\Usuario\Documents\Universitat\4rt Quatri\PLH\Practica4\word-embeddings\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [49]:
pearson_train, pearson_val, pearson_test = a.train(num_epochs=128)

Epoch 1/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 2.5821 - val_loss: 2.2390 - learning_rate: 0.0010
Epoch 2/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9768 - val_loss: 2.1459 - learning_rate: 0.0010
Epoch 3/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4046 - val_loss: 2.1272 - learning_rate: 0.0010
Epoch 4/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2322 - val_loss: 2.1087 - learning_rate: 0.0010
Epoch 5/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1967 - val_loss: 2.1315 - learning_rate: 0.0010
Epoch 6/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1992 - val_loss: 2.1281 - learning_rate: 0.0010
Epoch 7/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2112 - val_loss: 2.1486 - learning_rate: 0.0010
Epoch 8/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2004 - val_loss: 2.1854 - learning_rate: 0.0010
Epoch 9/128
29/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2020 
Epoch 9: ReduceLROnPlateau reducing learning rate to 0.000100

In [50]:
print(pearson_train, pearson_val, pearson_test )

0.910658877086481 0.4016002072532002 0.4383577289086792


0.8274677118644338 0.2021368295111794 0.17972441226355187


In [27]:
from scipy.stats import pearsonr
def compute_pearson(x_, y_):
    # Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
    y_pred = a.exec_model.predict(x_)
    # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
    correlation, _ = pearsonr(y_pred.flatten(), y_.flatten())
    return correlation
print(f"Correlación de Pearson (baseline-train): {compute_pearson(a.x_train, a.y_train)}")
print(f"Correlación de Pearson (baseline-validation): {compute_pearson(a.x_val, a.y_val)}")
print(f"Correlación de Pearson (baseline-test): {compute_pearson(a.x_test, a.y_test)}")



65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Correlación de Pearson (baseline-train): 0.6944484626760427
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Correlación de Pearson (baseline-validation): 0.2903849157142713
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Correlación de Pearson (baseline-test): 0.3850071528624018


# Finetuned model

In [ ]:
from transformers import pipeline, AutoTokenizer
from scipy.special import logit

model = 'projecte-aina/roberta-base-ca-v2-cased-sts'
tokenizer = AutoTokenizer.from_pretrained(model)
pipe = pipeline('text-classification', model=model, tokenizer=tokenizer)

def prepare(sentence_pairs):
    sentence_pairs_prep = []
    for s1, s2 in sentence_pairs:
        sentence_pairs_prep.append(f"{tokenizer.cls_token} {s1}{tokenizer.sep_token}{tokenizer.sep_token} {s2}{tokenizer.sep_token}")
    return sentence_pairs_prep

sentence_pairs = [("El llibre va caure per la finestra.", "El llibre va sortir volant."),
                  ("M'agrades.", "T'estimo."),
                  ("M'agrada el sol i la calor", "A la Garrotxa plou molt.")]

predictions = pipe(prepare(sentence_pairs), add_special_tokens=False)

# convert back to scores to the original 0 and 5 interval
for prediction in predictions:
    prediction['score'] = logit(prediction['score'])
print(predictions)


c:\Users\Usuario\Documents\Universitat\4rt Quatri\PLH\Practica4\word-embeddings\.venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Usuario\.cache\huggingface\hub\models--projecte-aina--roberta-base-ca-v2-cased-sts. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


[{'label': 'SIMILARITY', 'score': 2.1183004307689126}, {'label': 'SIMILARITY', 'score': 2.179974932297432}, {'label': 'SIMILARITY', 'score': 0.9511617858568939}]


In [ ]:
input_pairs = [(e["sentence1"], e["sentence2"]) for e in dataset["train"]]
y_train = [e["label"] for e in dataset["train"]]
input_pairs_val = [(e["sentence1"], e["sentence2"]) for e in dataset["validation"]]
y_val = [e["label"] for e in dataset["validation"]]
input_pairs_test = [(e["sentence1"], e["sentence2"]) for e in dataset["test"]]
y_test = [e["label"] for e in dataset["test"]]

In [ ]:
def prepare(sentence_pairs):
    sentence_pairs_prep = []
    for s1, s2 in sentence_pairs:
        sentence_pairs_prep.append(f"{tokenizer.cls_token} {s1}{tokenizer.sep_token}{tokenizer.sep_token} {s2}{tokenizer.sep_token}")
    return sentence_pairs_prep



def compute_pearson(input, y):
    # Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
    predictions = pipe(prepare(input), add_special_tokens=False)

    for prediction in predictions:
        prediction['score'] = logit(prediction['score'])

    y_pred = np.array([pred['score']for pred in predictions])
    # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
    correlation, _ = pearsonr(y_pred.flatten(), np.array(y).flatten())
    return correlation

print(f"Correlación de Pearson (baseline-test): {compute_pearson(input_pairs_test, y_test)}")

Correlación de Pearson (baseline-test): 0.781988582023819
